**annotation automatique de l'expertise avec EmoBert et recalibrage des scores avec listes lexicales :**

Ce notebook permet d'annoter automatique le corpus préalablement corrigé et découpé en extraits. Les étapes sont les suivantes :

1. annotation automatique de tous les segments avec EmoBert (modèle en libre accès sur la plateforme Hugging Face via : https://huggingface.co/astrosbd/french_emotion_camembert)
2. On produit un score émotionnel en additionnant tous les scores de chaque catégorie émotionnelle, en excluant la catégorie 'neutral' : on obtient un score de l'intensité émotionnelle. On note que les labels 'anger' et 'sad' obtiennent souvent des scores très haut.
3. Pour résoudre le problème de surévaluation des labels 'sad' et 'anger, on pondère les scores par label pour avoir des sommes plus harmonisées.
4. on fait une première classification des extraits selon les étiquettes faible (1), moyen (2), et fort (3), qui représente l'intensité émotionnelle de l'extrait.
5. On produit des listes de vocabulaire, qui correspondent aux classes moyen et faible : si l'extrait contient un mot ou une expression d'une liste, il est automatiquement classé selon le score correspondant.

Note : notebook rédigé dans Visual Studio Code.

On commence par les importations nécessaires :

In [1]:
!python -m pip install transformers


In [2]:
import transformers


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [4]:
pip install tiktoken


In [5]:
!pip install --upgrade --no-cache-dir sentencepiece


Attention, beaucoup de problèmes de compatibilité entre certaines versions de Python ou certains packages et Protobuf :

In [6]:
pip install protobuf


In [7]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [8]:
import torch

On renseigne notre token hugging face :

In [ ]:
from huggingface_hub import login
login("") #mettre son propre identifiant

On importe le modèle :

In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("astrosbd/french_emotion_camembert", use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained("astrosbd/french_emotion_camembert")
pipe = pipeline("text-classification", model="astrosbd/french_emotion_camembert")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/374 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Device set to use cpu


A présent, il faut faire une boucle qui permet de récupérer chaque score de probabilité pour chaque étiquette, afin de pouvoir calculer un score d'intensité émotionnelle.
On commence par visualiser quel chiffre correspond à quelle étiquette :

In [11]:
print(model.config.id2label)

{0: 'sad', 1: 'fear', 2: 'anger', 3: 'neutral', 4: 'surprise', 5: 'joy'}


à présent, on peut définir l'objet id2label, dictionnaire joignant chaque chiffre à l'étiquette :

In [12]:
id2label = {
    0: "sad",
    1: "fear",
    2: "anger",
    3: "neutral",
    4: "surprise",
    5: "joy"
}

On réalise un test pour vérifier qu'on arrive bien à extraire le score pour chaque étiquette :

In [13]:
text = "Et malheureusement, l'écologie et le capitalisme, ça va pas ensemble. C'est le problème. Après, il y a un parti pris sur la question des sciences que je trouve assez intéressant. Parce que moi, je connais qu'une seule science dure, une seule qui soit vraiment exacte. C'est les mathématiques."

# Préparation du texte
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Prédiction
outputs = model(**inputs)

# Normalisation des logits avec softmax
probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]

# Affichage des scores pour chaque label
for idx, score in enumerate(probabilities):
    label = id2label.get(idx, f"label_{idx}")
    print(f"{label}: {score.item():.4f}")

# Label prédit
predicted_label = id2label[probabilities.argmax().item()]
print(f"\nPredicted label: {predicted_label}")

sad: 0.0125
fear: 0.0013
anger: 0.0018
neutral: 0.9818
surprise: 0.0013
joy: 0.0012

Predicted label: neutral


On lance l'annotation automatique avec emoBert sur notre corpus :

In [15]:
import pandas as pd

In [18]:
mon_path = '/content/annotations_finales_corrigees.csv' #changer le path

On parse notre fichier :

In [19]:
mon_df= pd.read_csv(mon_path, sep=';', on_bad_lines='skip')

On ne prend que la colonne content :

In [22]:
textes=mon_df['textes']

on initialise une boucle for, qui va itérer sur chaque ligne du csv que l'on a en entrée :

1. On produit une annotation

In [23]:
scores = []
scores_echelle = []
scores_par_label = []

In [25]:
max_len = tokenizer.model_max_length

# Traitement
for element in textes:
    try:
        inputs = tokenizer(element, return_tensors="pt", padding=True, truncation=True, max_length=max_len)
        outputs = model(**inputs)

        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]

        label_scores = {id2label[idx]: prob.item() for idx, prob in enumerate(probabilities)}
        score_par_label = ", ".join([f"{label}: {score:.4f}" for label, score in label_scores.items()])

        predicted_idx = probabilities.argmax().item()
        predicted_label = id2label[predicted_idx]
        predicted_score = probabilities[predicted_idx].item()

        scores.append(predicted_score)
        scores_echelle.append(predicted_label)
        scores_par_label.append(score_par_label)

    except Exception as e:
        print(f"An error occurred: {e}")
        scores.append(None)
        scores_echelle.append(None)
        scores_par_label.append(None)

print(scores)
print(scores_echelle)
print(scores_par_label)

[0.9697827100753784, 0.9560182690620422, 0.9723992347717285, 0.9494917988777161, 0.9605324268341064, 0.9815760850906372, 0.9292305111885071, 0.9726178050041199, 0.4311428368091583, 0.37845781445503235, 0.9292305111885071, 0.9324951767921448, 0.9727223515510559, 0.9700973033905029, 0.6258091926574707, 0.7829159498214722, 0.9292305111885071, 0.9677578210830688, 0.849810779094696, 0.9810807108879089, 0.9774477481842041, 0.9502108693122864, 0.9251375794410706, 0.6104348301887512, 0.9550800919532776, 0.9544183611869812, 0.9887120723724365, 0.9550800919532776, 0.9804553389549255, 0.9750370979309082, 0.8205626606941223, 0.9663636088371277, 0.5604692101478577, 0.9572625756263733, 0.5794927477836609, 0.9961214661598206, 0.987373411655426, 0.9804553389549255, 0.9833694100379944, 0.9626448750495911, 0.9259632229804993, 0.9737729430198669, 0.9804553389549255, 0.8283674716949463, 0.8405600786209106, 0.888526201248169, 0.7297292947769165, 0.9411497712135315, 0.9667065739631653, 0.9720969796180725, 0

Si le résultat est statisfaisant, on ajoute les prédictions à notre dataframe :

In [31]:
mon_df["label_predit"] = scores_echelle         # Label (émotion) prédit
mon_df["score_par_label"] = scores_par_label

A présent, on calcule le score émotionnel, en additionnant le score de chaque label excepté neutral :

In [32]:
import re

In [33]:
def sum_except_neutral(score_str):
    # Cherche toutes les paires label: score
    pairs = re.findall(r'(\w+): ([0-9.]+)', score_str)
    # Additionne tous les scores sauf celui avec le label 'neutral'
    return sum(float(score) for label, score in pairs if label != 'neutral')

on applique la fonction sur le contenu de la colonne 'score_par_label', en créant une nouvelle colonne 'sum_without_neutral' :

In [34]:
mon_df['sum_without_neutral'] = mon_df['score_par_label'].apply(sum_except_neutral)

In [35]:
mon_df

,mes_scores_emo,scores_mat_emo,mes_scores_expert,scores_mat_expert,textes,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,sum_without_neutral,label_predit,score_par_label
0,3,3,2,2,Al Gore,candidat d�mocrate � la pr�sidentielle am�ric...,et Al Gore qui nous promettait �a pour 2013,et puis c'est aussi un discours que tenait Jo...,mais qui a travaill� aussi pour l'administrat...,puis pour l'administration Biden sur les affa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0302,neutral,"sad: 0.0070, fear: 0.0020, anger: 0.0010, neut..."
1,1,1,2,2,Ce n'est pas tout � fait un organisme scientif...,c'est un organisme politique. En gros,il est une �manation de l'ONU,en particulier de deux gros d�partements,en quelque sorte,dans de grandes �manations. C'est le PNUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0440,neutral,"sad: 0.0039, fear: 0.0022, anger: 0.0010, neut..."
2,3,3,2,1,Aujourd'hui,� la date,on parle exactement de �a. C'est quand m�me h...,parce que ce n'est pas ce qu'on entend partout,quoi,et qu'on dit : voil�,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0276,neutral,"sad: 0.0088, fear: 0.0022, anger: 0.0013, neut..."
3,2,3,2,1,C'est-�-dire que si on veut faire quelque chos...,la fa�on de voir le monde dans nos rapports d...,c'est plus d'humain et accepter l'id�e qu'on ...,cher Sylvain. On peut dire que c'est la terre...,exactement. C'est insupportable et �a n'est p...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0506,neutral,"sad: 0.0160, fear: 0.0022, anger: 0.0028, neut..."
4,1,1,3,3,Maintenant,il faut bien comprendre une chose,c'est que fondamentalement,l'atmosph�re,le r�chauffement de l'atmosph�re ne peut pas ...,il fait trop froid. Donc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0394,neutral,"sad: 0.0138, fear: 0.0025, anger: 0.0016, neut..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1,2,3,3,Il ne l'a jamais �t�. J'ai souvent entendu dir...,selon lequel les hommes seraient en train de ...,en tant que scientifique,je pense que c'est faux. Le r�chauffement cli...,le GIEC,le groupe intergouvernemental sur l'�volution...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5916,neutral,"sad: 0.3486, fear: 0.0174, anger: 0.1880, neut..."
92,2,1,1,2,C'est comme �a,c'est une faiblesse de l'�me humaine. C'est b...,mais c'est malheureux. Il ne faut pas perdre ...,on n'a pas besoin de r�veiller 100 % des gens...,c'est fabuleux.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1946,neutral,"sad: 0.1709, fear: 0.0033, anger: 0.0070, neut..."
93,1,1,3,3,Pourquoi dites-vous que ces d�penses pour r�du...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9527,anger,"sad: 0.2293, fear: 0.0138, anger: 0.6863, neut..."
94,1,1,3,3,"Alors, quels sont les param�tres qui sont vis�...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0207,neutral,"sad: 0.0039, fear: 0.0036, anger: 0.0017, neut..."


Après analyse manuelle, on repère que le modèle a tendance à surévaluer le label 'anger'. Nous produisons donc des sommes recalibrées :
1. on calcule la moyenne des scores de chaque label.
2. pour chaque segment de texte, on prend le score donné par le modèle pour chaque label, et on le divise par la moyenne des scores du label.
3. On additionne ces nouveaux scores, pour avoir une somme recalibrée.
4. On transforme ces sommes recalibrées en classe (1 pour faible, 2 pour moyen ou 3 pour fort).

on calcule la moyenne pour chaque label, et recalcule les sommes recalibrées :

In [37]:
# Calcule la moyenne par label sur tout le corpus
from collections import defaultdict

sums = defaultdict(float)
counts = defaultdict(int)

for row in mon_df['score_par_label']:
    pairs = re.findall(r'(\w+): ([0-9.]+)', row)
    for label, score in pairs:
        score = float(score)
        sums[label] += score
        counts[label] += 1

moyennes = {label: sums[label]/counts[label] for label in sums}

# Fonction pour recalibrer un score
def recalibrated_sum(score_str):
    pairs = re.findall(r'(\w+): ([0-9.]+)', score_str)
    return sum(float(score)/moyennes[label] for label, score in pairs if label != 'neutral')

mon_df['sum_recalibree'] = mon_df['score_par_label'].apply(recalibrated_sum)

On classe ces sommes recalibrées en trois catégories (1, 2 ou 3) ; les seuils suivants ont été déterminés en visualisant la répartition des sommes recalibrées et en tâtonnant :

In [38]:
#optionnel: on regarde la répartition :

mon_df['sum_recalibree'].describe()

,sum_recalibree
count,96.000000
mean,5.000000
std,7.384711
min,0.404860
25%,0.689805
50%,1.773574
75%,7.185250
max,47.699188


In [40]:
# 1. Fonction ajustée avec nouveaux seuils
def classer_recalibree(score):
    if score < 4:
        return 1  # Faible
    elif score < 18:
        return 2  # Moyen
    else:
        return 3  # Fort

# 2. Fonction principale pour attribuer l'intensité
def attribuer_intensite(row):
    if row['label_predit'] == 'neutral':
        return 1
    else:
        return classer_recalibree(row['sum_recalibree'])

# 3. Application de la fonction ligne par ligne
mon_df['intensite_emotion'] = mon_df.apply(attribuer_intensite, axis=1)

dernière étape : On remarque que malgré nos recalibrages, le modèle manque de sensibilité et ne détecte pas la valence attribuée au vocabulaire spécialisé, ni au vocabulaire injurieux. On utilise une méthode par dictionnaire pour forcer la classification vers certaines classes lorsque ce vocabulaire est présent.

Le code suivant détermine les listes de vocabulaire, pour la classe moyenne et la classe forte. Ensuite, une boucle nettoie le texte pour retirer les apostrophes anormales, enlever la poncctuation, et normaliser les espaces. Enfin, on applique les fonctions à la bonne colonne de notre csv.

In [ ]:
# 1. Listes de mots-clés
motifs_moyenne = ["propagande", "arrêtez", "punks", "punk", "alarmiste", "bourrage de crâne", "endoctrinement",
    "climato-alarmiste", "mensonges médiatiques", "pseudo-science", "idéologie", "matraquage", "baratin",
    "élite", "panique", "panique climatique", "instrumentalisation", "folie", "lamentable", "pour que ça colle", "traffiquer", "truquer", "truqué",
    "catastrophisme", "catastrophe", "bête", "bêtise", "manipulé", "n'existe pas", "n’y a pas de preuve", "bidon", "fake", "climatomaniaque",
    "illusion", "mensongère", "intox", "fabriqué", "ruine", "peur", "guerre psychologique", "minable", "ruine", "cataclysme", "cataclysmique", "idiot", "idiots", "bien pensance", "bien pensant", "délirant", "délirante", "mensonge", "sachant", "sachants", "apocalypse", "apocalyptique", "manipulation", "marre", "ridicule", "corruption", "desastre", "desastreux", "desastreuse", "contrôler", "mensonger", "terrible", "dangereux", "danger", "colère"]
motifs_forte = ["menteurs", "criminels", "coupable", "dictature verte", "écotyrannie", "détruire l’humanité",
    "lavage de cerveau", "controler totalement", "tuer l'humanité", "crime", "haine", "manipulateurs", "collabos", "dictature",
    "ils veulent nous", "ils cherchent à", "foutre en l’air", "plan diabolique", "salauds", "baratin", "ils sont beaux",
    "on nous prend pour des cons", "infernal", "infernale", "terrorisme", "fascisme", "fasciste", "totalitariste", "totalitaire", "couillonner", "contrôler totalement", "suicide", "gens dangereux", "dirigeants dangereux", "on vous a ruiné", "ces pourris", "ces salauds", "délire", "délires", "c’est une arnaque", "escrocs", "pourriture","dégueulasse", "détruit le monde", "indignité", "les dénoncer", "indigne", "atroce", "atrocité", "atrocités", "horrible", "vous contrôler", "nous contrôler", "c'est terrible",  "foutaise", "foutaises", "honteux", "inadmissible", "scandaleux", "trahison", "colère", "c’est une honte", "merde", "merdes", "merdeux", "putain", "connard", "connards"]

#voici la liste de mots clefs alternative, plus adaptée au corpus témoin (à décommenter si on veut l'utiliser) :

#motifs_moyenne = ["urgence", "menace", "préoccupant", "inquiétant", "fragilité", "crise climatique","réchauffement inquiétant", "situation alarmante", "alerte", "montée des eaux","sécheresse", "vagues de chaleur", "fonte des glaces", "niveaux records", "pollution","risque élevé", "hausse préoccupante", "réalité du changement", "effets visibles","dérèglement climatique", "fréquence accrue", "périodes caniculaires", "perturbations","pression sur les ressources", "impact grandissant", "augmentation des catastrophes","risques futurs", "phénomènes extrêmes", "changement rapide", "conséquences graves"]
#motifs_forte = ["effondrement", "catastrophe", "irréversible", "point de non-retour", "disparition","planète en danger", "scénario catastrophe", "fin de l’humanité", "danger mortel","survie de l’espèce", "anéantissement", "menace existentielle", "urgence absolue","extinction massive", "monde invivable", "terre inhabitable", "en danger de mort","plus aucun espoir", "trop tard", "plus de retour possible", "situation désespérée","effets dévastateurs", "avenir sombre", "avenir compromis", "planète condamnée","drame climatique", "chaos climatique", "climat hors de contrôle", "urgence vitale"]

def nettoyer(texte):
    texte = texte.lower()
    texte = texte.replace("’", "'")  # apostrophe courbe
    texte = re.sub(r"[^\w\s']", " ", texte)  # enlever ponctuation sauf apostrophe
    texte = re.sub(r"\s+", " ", texte).strip()  # normalise les espaces
    return texte

 #3. Nettoyage du contenu
mon_df['textes'] = mon_df['textes'].astype(str).apply(nettoyer)

# 4. Fonction de surclassement selon les nouvelles règles
def surclasser_si_mot_present(row):
    score = row['intensite_emotion']
    content = row['textes']

    if score in [1, 2] and any(mot in content for mot in motifs_forte):
        return 3
    elif score == 1 and any(mot in content for mot in motifs_moyenne):
        return 2
    return score  # sinon, on garde la valeur initiale

# 5. Application de la fonction
mon_df['score_vf'] = mon_df.apply(surclasser_si_mot_present, axis=1)

enfin, on charge le document final :

In [ ]:
mon_df.to_csv("fichier_complet_annotations_emotion.csv", index=False)